In [1]:
# Testando API YouTube
import json
import pandas as pd
import seaborn as sns
import numpy as np
import requests
import csv 
from datetime import datetime, timezone

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from googleapiclient.discovery import build

In [6]:
# Colocando nossa chave da API do YouTube
import os
from getpass import getpass

api_key = getpass("Insira aqui sua API key do Youtube:")


In [7]:
# Função para buscar os vídeos em alta no YouTube e armazenar os dados em um DF
def videos_em_alta(n_videos: int, region: str):

    youtube = build('youtube', 'v3', developerKey=api_key)
    response = youtube.videos().list(
        part="snippet, statistics", 
        chart="mostPopular", 
        regionCode=region, 
        maxResults=n_videos,
            ).execute()
    videos = response.get("items", []) 

    #caso não há vídeos em alta exibirá a mensagem
    if not videos:
        print("Nenhum vídeo em alta foi encontrado! Ou informações não estão disponíveis")

    # dicionário para salvar as informações
    video_dict = {}

    for video in videos:
        

    # Extraindo as informações 
        video_id = video["id"]
        title = video["snippet"]["title"]
        views = video["statistics"]["viewCount"]
        likes = video["statistics"]["likeCount"]
        comments = video["statistics"]["commentCount"]
        published_at = video["snippet"]["publishedAt"]
        link_video = f"https://www.youtube.com/watch?v={video_id}"

        id = video["snippet"]['categoryId']
        response_cat = youtube.videoCategories().list(part="snippet",  id=id).execute()
        category = response_cat.get("items", [])[0]
        category_title = category["snippet"]["title"]

    # Armazenando as infos no dicionário
        video_dict[title] = {
            "views": views,
            "likes": likes,
            "comentarios": comments,
            "publicacao": published_at,
            "link_video": link_video,
            "categoria": category_title
        }
    # Salvando os dados em um DataFrame
    df = pd.DataFrame.from_dict(video_dict, orient="index")
    df = df.reset_index().rename(columns={'index': 'video'})

    # Convertendo a coluna 'publicacao' para datetime e criando uma nova coluna com o tempo postado
    current_time = datetime.now(timezone.utc)

    df['publicacao'] = pd.to_datetime(df['publicacao'])
    df["horas-publicacao"] = (current_time - df["publicacao"]).dt.total_seconds() / 3600
    df["horas-publicacao"] = df["horas-publicacao"].round()
    return df


In [8]:
# função para fazer update diário dos novos vídeos
def update_df(df, df_up):
    df = pd.concat([df,df_up])
    return df.reset_index(drop=True)


In [11]:
# Abrindo e salvando o df em uma variável
df = videos_em_alta(10, 'BR')
df

,video,views,likes,comentarios,publicacao,link_video,categoria,horas-publicacao
0,New York Knicks x Boston Celtics | AO VIVO COM...,511021,56222,15,2023-11-14 03:09:00+00:00,https://www.youtube.com/watch?v=FgoRRxDTmGk,Sports,11.0
1,AO VIVO E COM IMAGENS: SANTOS X SÃO PAULO | SE...,466541,14720,51,2023-11-14 02:27:13+00:00,https://www.youtube.com/watch?v=x47X1yJJoNM,Sports,12.0
2,"Tchau Obrigado 2 - MC Ryan SP, Kadu, IG, MC GP...",353158,39675,2775,2023-11-13 22:00:10+00:00,https://www.youtube.com/watch?v=8dfSth3DrO4,Music,16.0
3,skibidi toilet 67 (part 3),17358836,1346036,148514,2023-11-14 02:00:09+00:00,https://www.youtube.com/watch?v=vIgnPmmcSXs,Entertainment,12.0
4,DE FRENTE COM BLOGUEIRINHA: ESPECIAL FINAL DRA...,197641,17921,473,2023-11-14 01:17:57+00:00,https://www.youtube.com/watch?v=h5Q1BE2vw8g,Entertainment,13.0
5,Mulher de líder de facção criminosa é recebida...,278230,10181,3184,2023-11-13 22:28:09+00:00,https://www.youtube.com/watch?v=9VnS0EjMGHw,News & Politics,16.0
6,Alexandre Correa afirma que mentiu sobre agres...,200393,5576,437,2023-11-13 22:00:46+00:00,https://www.youtube.com/watch?v=3-aBv1AoMTo,Entertainment,16.0
7,CHARLLES TROLA TÕE COM UM SAPO NO SEU ANIVERSÁ...,193546,22434,432,2023-11-13 21:30:05+00:00,https://www.youtube.com/watch?v=dHgw6IIqswI,Comedy,17.0
8,#OwRangel 14 - FAZENDO M3RD4 SOZINHO,97736,14488,509,2023-11-13 15:01:50+00:00,https://www.youtube.com/watch?v=vOobgv3etm8,Comedy,23.0
9,TET A TET - PRISCILA JÚ,209436,31728,1506,2023-11-12 22:18:15+00:00,https://www.youtube.com/watch?v=PuCqKuq7cos,People & Blogs,40.0


In [13]:
df_us = videos_em_alta(10, 'US')


In [14]:

df = update_df(df, df_us)

In [ ]:
df

,video,views,likes,comentarios,publicacao,link_video,categoria,horas-publicacao
0,"VINI JR CRAVA, RODRYGO FAZ GOLAÇO E REAL SE CL...",1359713,73549,1650,2023-11-08 22:08:04+00:00,https://www.youtube.com/watch?v=8bvY-YpAeQs,Sports,18.0
1,FLAMENGO 3 X 0 PALMEIRAS | MELHORES MOMENTOS |...,1525278,59646,4142,2023-11-09 02:28:15+00:00,https://www.youtube.com/watch?v=0R1AU9ZL3Y0,Sports,14.0
2,LIGA MFL: GRANDE FINAL| NOISE - CRIAS - FAZ O ...,621269,51287,81,2023-11-09 03:26:37+00:00,https://www.youtube.com/watch?v=s5eiOSaIu7Y,Gaming,13.0
3,MUDANÇA RADICAL AQUI NO SÍTIO - NOVA FASE - RE...,288951,35153,4309,2023-11-08 19:04:49+00:00,https://www.youtube.com/watch?v=rAXw2WOOJd4,Entertainment,21.0
4,SEM VIRADA HOJE! OLHA COMO O CRAQUE NETO REAGI...,130991,6094,517,2023-11-09 02:47:40+00:00,https://www.youtube.com/watch?v=fkltXNCtGdc,Sports,13.0
5,VOLTEI COM A MARIA EMANUELLY!!?,184237,16248,845,2023-11-08 23:01:08+00:00,https://www.youtube.com/watch?v=gP03BLp3XYc,People & Blogs,17.0
6,GTA 6 oficialmente ANUNCIADO e trailer em DEZE...,226843,40251,1066,2023-11-08 14:22:25+00:00,https://www.youtube.com/watch?v=W3Tsyf_OZEw,Gaming,26.0
7,A Ludmilla e o Hino Nacional (HUMOR),241411,18304,846,2023-11-08 14:28:05+00:00,https://www.youtube.com/watch?v=KbbjAHzDSmk,Comedy,26.0
8,TÕE TROCA O CARRO DE TIRINGA EM UM BAR CHEIO D...,204168,23183,399,2023-11-08 21:06:19+00:00,https://www.youtube.com/watch?v=VDWNF0cU8Tk,Comedy,19.0
9,GTA 6 É OFICIALMENTE ANUNCIADO PELA ROCKSTAR,167248,32664,3043,2023-11-08 19:48:52+00:00,https://www.youtube.com/watch?v=ha4lbP4HSck,Gaming,20.0


In [15]:
# converendo as colunas 'views', 'likes', 'comentarios' para int
df['views'] = df['views'].astype(int)
df['likes'] = df['likes'].astype(int)
df['comentarios'] = df['comentarios'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   video             20 non-null     object             
 1   views             20 non-null     int32              
 2   likes             20 non-null     int32              
 3   comentarios       20 non-null     int32              
 4   publicacao        20 non-null     datetime64[ns, UTC]
 5   link_video        20 non-null     object             
 6   categoria         20 non-null     object             
 7   horas-publicacao  20 non-null     float64            
dtypes: datetime64[ns, UTC](1), float64(1), int32(3), object(3)
memory usage: 1.1+ KB


In [16]:
# calculando a quantidade de views por hora
df["views/hour"] = (df["views"]/df["horas-publicacao"]).round(2)
# excluindo os dados duplicados
df = df.drop_duplicates(subset='video').reset_index(drop=True)
df

,video,views,likes,comentarios,publicacao,link_video,categoria,horas-publicacao,views/hour
0,New York Knicks x Boston Celtics | AO VIVO COM...,511021,56222,15,2023-11-14 03:09:00+00:00,https://www.youtube.com/watch?v=FgoRRxDTmGk,Sports,11.0,46456.45
1,AO VIVO E COM IMAGENS: SANTOS X SÃO PAULO | SE...,466541,14720,51,2023-11-14 02:27:13+00:00,https://www.youtube.com/watch?v=x47X1yJJoNM,Sports,12.0,38878.42
2,"Tchau Obrigado 2 - MC Ryan SP, Kadu, IG, MC GP...",353158,39675,2775,2023-11-13 22:00:10+00:00,https://www.youtube.com/watch?v=8dfSth3DrO4,Music,16.0,22072.38
3,skibidi toilet 67 (part 3),17358836,1346036,148514,2023-11-14 02:00:09+00:00,https://www.youtube.com/watch?v=vIgnPmmcSXs,Entertainment,12.0,1446569.67
4,DE FRENTE COM BLOGUEIRINHA: ESPECIAL FINAL DRA...,197641,17921,473,2023-11-14 01:17:57+00:00,https://www.youtube.com/watch?v=h5Q1BE2vw8g,Entertainment,13.0,15203.15
5,Mulher de líder de facção criminosa é recebida...,278230,10181,3184,2023-11-13 22:28:09+00:00,https://www.youtube.com/watch?v=9VnS0EjMGHw,News & Politics,16.0,17389.38
6,Alexandre Correa afirma que mentiu sobre agres...,200393,5576,437,2023-11-13 22:00:46+00:00,https://www.youtube.com/watch?v=3-aBv1AoMTo,Entertainment,16.0,12524.56
7,CHARLLES TROLA TÕE COM UM SAPO NO SEU ANIVERSÁ...,193546,22434,432,2023-11-13 21:30:05+00:00,https://www.youtube.com/watch?v=dHgw6IIqswI,Comedy,17.0,11385.06
8,#OwRangel 14 - FAZENDO M3RD4 SOZINHO,97736,14488,509,2023-11-13 15:01:50+00:00,https://www.youtube.com/watch?v=vOobgv3etm8,Comedy,23.0,4249.39
9,TET A TET - PRISCILA JÚ,209436,31728,1506,2023-11-12 22:18:15+00:00,https://www.youtube.com/watch?v=PuCqKuq7cos,People & Blogs,40.0,5235.90


In [17]:
# Data de hoje 
import datetime

date_today = datetime.date.today()
date_today


datetime.date(2023, 11, 14)

In [ ]:
# salvando o arquivo em um csv

df.to_csv(f'dados/top_10_youtube_{date_today}.csv', sep=';',index=False)